In [1]:
from pathlib import Path

from pandas import (
    read_csv,
    to_datetime,
)

from post_processing.dataclass.data_aplose import DataAplose
from post_processing.utils.fpod_utils import resample_dpm, feeding_buzz, txt_folder, build_range

### Load data


In [2]:
yaml_file = Path(r"resource\CPOD-FPOD_yaml.yml")
data_list = DataAplose.from_yaml(file=yaml_file)

fb_files = Path(r"U:\fb_fpod_cetiroise_c")  #Path to your click details folder.
d_beg_end = read_csv(r"U:\Deb_Fin_CETIROISE.csv")

### Data metric
Here you need to choose the format in which you want to visualise the positive detections. This aims to replace the functionalities available in Chelonia's softwares.

In [3]:
frq = "D" #This argument will determine the format in which you want to visualise your data. Use "D", "h" or "10min".

In [4]:
resamp = resample_dpm(data_list.df, frq=frq, extra_columns=["deploy.name"])

### Feeding buzzes processing
Use "Marsouin" or "Commerson" to get different ICI processing.

In [5]:
fb_all = txt_folder(fb_files)
fb_all = feeding_buzz(fb_all, "Marsouin")

fb_all["start_datetime"] = fb_all["start_datetime"].dt.floor(frq)
fb = fb_all.groupby("start_datetime")["Foraging"].sum().reset_index()
fb["start_datetime"] = to_datetime(fb["start_datetime"], utc=True)

dpm_fb = resamp.merge(fb, on="start_datetime", how="left")

In [6]:
d_0 = build_range(d_beg_end, frq)
d_tot = d_0.merge(dpm_fb, on=["start_datetime", "deploy.name"], how="left")

In [7]:
d_tot[["DPM","Foraging"]] = d_tot[["DPM","Foraging"]].fillna(0)